# 非线性规划

## 模型

类似于线性规划，但是表达式含**非线性部分**，如 $x^{2}$, $e^{x}$, $\ln{x}$ 等

## 实现

❗ 注意 python 中非线性规划的约束 `constraints` 是大于号方向 $g(x) \geq 0$

In [19]:
import numpy as np
from scipy.optimize import minimize

#! 修改
result = minimize(
    # 初始值
    x0  = [1, 2],
    # 目标表达式
    fun = lambda x, a, b: a * x[0] ** 2 + b * x[1] ** 2,
    # 目标表达式参数
    args = (1, 2), # a = 1, b = 2
    # 取值范围约束
    bounds = [
        [0, np.inf],
        [0, np.inf],
    ],
    # 表达式约束
    constraints = [
        { # x + y <= 1
            'type': 'ineq',
            'fun': lambda x: 1 - (x[0] + x[1])
        },
        { # x^2 + y^2 = 1
            'type': 'eq',
            'fun': lambda x: x[0] ** 2 + x[1] ** 2
        }
    ],
)

# print(result)
print(f"求解{result.success and "成功" or "失败"}!")
print("最优值: ", result.fun)
print("解向量: ", result.x)
print("优化中止信息: ", result.message)

求解成功!
最优值:  1.298862180714911e-07
解向量:  [3.60397306e-04 1.05813740e-20]
优化中止信息:  Optimization terminated successfully


### 模拟退火

关注全局最优

更多全局最优算法见 [scipy 官网](https://docs.scipy.org/doc/scipy/reference/optimize.html#global-optimization)

In [20]:
import numpy as np
from scipy.optimize import dual_annealing

#! 修改
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.dual_annealing.html
result = dual_annealing(
    # 初始值
    x0  = [1, 2],
    # 目标表达式
    func = lambda x, a, b: a * x[0] ** 2 + b * x[1] ** 2,
    # 目标表达式参数
    args = (1, 2), # a = 1, b = 2
    # 取值范围约束
    # 不允许无穷
    bounds = [
        (0, 1000000),
        (0, 1000000),
    ],
    # 表达式约束
    minimizer_kwargs = {
        'constraints':  [
            { # x + y <= 1
                'type': 'ineq',
                'fun': lambda x: 1 - (x[0] + x[1])
            },
            { # x^2 + y^2 = 1
                'type': 'eq',
                'fun': lambda x: x[0] ** 2 + x[1] ** 2
            }
        ],
    },
    # 最大迭代次数
    maxiter = int(1e3),
)

# print(result)
print(f"求解{result.success and "成功" or "失败"}!")
print("最优值: ", result.fun)
print("解向量: ", result.x)
print("优化中止信息: ", result.message)

求解成功!
最优值:  1.2076512741469584e-07
解向量:  [0.0003072  0.00011488]
优化中止信息:  ['Maximum number of iteration reached']


### Gurobipy

`Gurobipy` 在大型数据的计算中更快，但是目标类型和约束类型比较受限，只支持最高二次

In [ ]:
import gurobipy as grb

model = grb.Model('My_Problem')

# 创建多下标变量 X[3, 4, 5]
X = model.addVars(3, 4, 5, lb=0.0, ub=10.0, vtype=grb.GRB.CONTINUOUS)
# 创建离散变量
y = model.addVar(ub=5, vtype=grb.GRB.INTEGER)
ey = model.addVar(); model.addGenConstrExp(y, ey) # ey = e^y
# 创建布尔变量
z = model.addVar(vtype=grb.GRB.BINARY)

# 创建目标函数
model.setObjective(X.sum() + y * z, grb.GRB.MAXIMIZE)

# 添加约束
model.addConstr(X.sum() <= y)

model.setParam('MIPGap', 0.01)

model.optimize()

if model.Status == grb.GRB.OPTIMAL:
    print("最优解: ", model.ObjVal)
    for i in range(3):
        for j in range(4):
            for k in range(5):
                print(f"X[{i}, {j}, {k}] = {X[i, j, k].X}")
    print(f"y = {y.X}")
    print(f"z = {z.X}")

Set parameter MIPGap to value 0.01
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13980HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
MIPGap  0.01

Optimize a model with 1 rows, 62 columns and 61 nonzeros
Model fingerprint: 0xac715989
Model has 1 quadratic objective term
Variable types: 60 continuous, 2 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 5.0000000
Presolve removed 1 rows and 62 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 32 available processor